#### Agents SDK Course

## Tracing

Like all LLM frameworks, Agents SDK provides a way to pass both user and system prompts to the LLM. As a basic example, we will dive into how to use this functionality

Firstly we need to get a `OPENAI_API_KEY` set up, for this you will need to create an account on [OpenAI](https://platform.openai.com/api-keys) and grab your API key

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass("OpenAI API Key: ")

### Tracing Basic Agent Prompting

First we need to create an agent, we will use the `Agent` class to create a basic agent.

In each `Agent` object we have some fundamental parameters, those are:
- `name`: The name of the agent
- `instructions`: The system prompt for the agent
- `model`: The model to use for the agent

The `instructions` parameter is where we will pass our system prompt, this will be used to guide the behavior of the agent


In [2]:
from agents import Agent # object class

prompt_agent = Agent(
    name="Prompt Agent", # name of the agent
    instructions="Speak like a pirate.", # system prompt
    model="gpt-4o" # model to use
)

Now we want to run the agent, Agents SDK provides a `Runner` object that will allow us to run the agent

However we first need to make sure tracing is active, to do this we need to also import `trace` from the Agents SDK library, then to start a trace we need to use the `with trace()`. Inside the function we can add additonal labels such as:
- `workflow_name`: Name given to the workflow
- `trace_id`: ID of the individual trace (recommended to use `util.gen_trace_id()` if using)
- `group_id`: ID given to a collective of traces
- `metadata`: Dictionary of data given with the workflow data
- `disabled`: If true it will not return a trace

In the `run` method we have the following parameters:
- `starting_agent`: The agent to start the conversation with
- `input`: The input to pass to the agent

In [3]:
from agents import Runner, trace # object class

query = "Write a one-sentence poem."

with trace(workflow_name="Prompt Trace", group_id="Tracing 101"):
    prompt_result = await Runner.run(
        starting_agent=prompt_agent, # agent to start the conversation with
        input=query # input to pass to the agent
    )

From our instructions *'Speak like a pirate.'* and our input *'Write a one-sentence poem.'* the agent will generate a response which we'd hope would be a one-sentence poem spoken like a pirate

In [4]:
print("Prompt Agent Result:", prompt_result.final_output)

Prompt Agent Result: On the crest o' waves under moonlit skies, me heart sails free 'neath starry eyes.


### Tracing Agent Tools

Again, we need to create an agent.

In this `Agent` object we have an additional parameter, this is:
- `tools`: The tools provided to the agent for usage.

In order to use this we need to aquire a tool to use, we can import `WebSearchTool` from the agents library, this tool will allow the `agent` object to search the web for results.

In [5]:
from agents import Agent, WebSearchTool

tool_agent = Agent(
    name="Tool Agent",
    instructions="You are a web search agent that searches the web for information on the user's query.",
    tools=[
        WebSearchTool()
    ],
    model='gpt-4o'
)

This time we can use the trace parameters to define the tool we have given to our agent within the metadata, this will provide better information in the trace.

In [6]:
query = "What are the current headlines in the news?"

with trace(workflow_name="Tool Trace", group_id="Tracing 101", metadata={"Tools": "WebSearchTool"}):
    tool_result = await Runner.run(
        starting_agent=tool_agent,
        input=query
    )

In [22]:
print("Tool Agent Result:", tool_result.final_output)

Tool Agent Result: Here are some of the latest news headlines as of April 24, 2025:

**U.S. News:**

- **Polar Vortex Grips Much of the U.S.:** A severe polar vortex is causing frigid temperatures across large portions of the United States, leading to disruptions and safety concerns. ([apnews.com](https://apnews.com/?utm_source=openai))

- **Minneapolis Agrees to Overhaul Police Training:** In response to the murder of George Floyd, Minneapolis has agreed to significant changes in police training and use-of-force policies. ([apnews.com](https://apnews.com/?utm_source=openai))

**International News:**

- **Strong Earthquake in Western China:** A powerful earthquake has struck western China, resulting in at least 53 fatalities and extensive damage. ([apnews.com](https://apnews.com/?utm_source=openai))

- **Gunmen Attack Bus in West Bank:** An attack on a bus in the occupied West Bank has left three people dead, escalating tensions in the region. ([apnews.com](https://apnews.com/?utm_sour